# Sciunit Cross-OS Reproducibility Demo on Chameleon

1. Create an **Ubuntu VM** to author a Sciunit experiment
2. Create a **Fedora VM** to reproduce the experiment
3. Use SSH to interact with both machines
4. Demonstrate cross-OS reproducibility using Sciunit

This notebook focuses only on **infrastructure setup**.
Sciunit commands are executed directly on the VMs


In [ ]:
import chi, os, time, datetime
from chi import lease
from chi import server
from chi import context
from chi import network

context.version = "1.0"
context.choose_project()
context.choose_site(default="KVM@TACC")

username = os.getenv("USER")


In [ ]:
exp_name = "sciunit_cross_os_demo"

ubuntu_server_name = f"{exp_name}-ubuntu-{username}"
centOS_server_name = f"{exp_name}-centOS-{username}"
lease_name = f"{exp_name}-lease-{username}"

print("Experiment name:", exp_name)


In [ ]:
l = lease.Lease(
    lease_name,
    duration=datetime.timedelta(hours=6)
)

l.add_flavor_reservation(
    id=server.get_flavor_id("m1.small"),
    amount=2
)

l.submit(idempotent=True)
l.show()


In [ ]:
ubuntu_image = "CC-Ubuntu22.04"

ubuntu_vm = server.Server(
    name=ubuntu_server_name,
    image_name=ubuntu_image,
    flavor_name=l.get_reserved_flavors()[0].name
)

ubuntu_vm.submit(idempotent=True)


In [ ]:
centOS_image = "CC-CentOS8-Stream"

centOS_vm = server.Server(
    name=centOS_server_name,
    image_name=centOS_image,
    flavor_name=l.get_reserved_flavors()[0].name
)

centOS_vm.submit(idempotent=True)


In [ ]:
ubuntu_vm.associate_floating_ip()
ubuntu_ip = ubuntu_vm.get_floating_ip()
print("Ubuntu VM IP:", ubuntu_ip)

In [ ]:
centOS_vm.associate_floating_ip()
centOS_ip = centOS_vm.get_floating_ip()
print("centOS VM IP:", centOS_ip)

In [ ]:
sg_list = network.list_security_groups(name_filter="allow-ssh")

if sg_list:
    sg = sg_list[0]
else:
    sg = network.SecurityGroup({
        "name": "allow-ssh",
        "description": "Enable SSH access"
    })
    sg.add_rule("ingress", "tcp", 22)
    sg.submit()

ubuntu_vm.add_security_group(sg.id)
centOS_vm.add_security_group(sg.id)


In [ ]:
ubuntu_vm.check_connectivity()
centOS_vm.check_connectivity()

print("Both Ubuntu and centOS VMs are ready.")


### SSH commands for connecting to VM

In [ ]:


print("SSH into Ubuntu (Sciunit author):")
print(f"ssh cc@{ubuntu_ip}")

print("SSH into centOS (Sciunit reproducer ):")
print(f" ssh cc@{centOS_ip}")


## Install Sciunit on the Ubuntu VM

The following commands must be run **inside the Ubuntu VM terminal**, not in Jupyter.
Sciunit operates at the operating-system level and must be installed in a real shell
environment to correctly capture execution provenance.

open a new terminal, from File >> New >> Terminal

SSH and connect with the remote ubuntu VM using the above "SSH commands for connecting to VM" step, take the first ssh cc@xxx.xxx.xx.xxx execute it in the new opened terminal

### Step 1: Verify Python and pip installation

```bash
python3 --version

pip --version



### (optional if pip not present) Step 2: Install pip

sudo apt update

sudo apt install -y python3-pip

#### check whether pip has installed or not using the following command

pip --version



### Step 3: Install Sciunit

run the following commands

git clone https://github.com/radiant-systems-lab/sciunit.git

cd sciunit

sudo apt install cmake

git checkout fix-sciunit-copy

pip install . 




### Step 4: Add user bin to PATH

echo 'export PATH=$HOME/.local/bin:$PATH' >> ~/.bashrc

source ~/.bashrc

#### check the version with the following command

sciunit --version

Expected output: sciunit2 0.4.post115.dev43584757

## Install Sciunit on the CentOS VM

The following commands must be run **inside the CentOS VM terminal**, not in Jupyter.
Sciunit operates at the operating-system level and must be installed in a real shell
environment to correctly capture execution provenance.

open a new terminal, from File >> New >> Terminal

SSH and connect with the remote centOS VM using the above "SSH commands for connecting to VM" step, take the second ssh cc@xxx.xxx.xx.xxx and execute it in the new opened terminal



### Step 1: Verify Python and pip installation

```bash
python3 --version

pip --version
```






### (optional if pip not present) Step 2: Install pip  

sudo dnf module enable -y python36

sudo dnf install -y python3-pip

#### check if pip installed with following command

pip --version

### Step 3: Install Sciunit

pip install --user sciunit2

#### check the version to confirm sciunit installation once done

sciunit --version

Expected output: sciunit2 0.4.post115.dev43584757

## Cleanup 

In [ ]:
ubuntu_vm.delete()
centOS_vm.delete()
l.delete()

print("Cleanup commands prepared (commented out).")


_______________________________________________________________________________